In [25]:
from urllib.request import urlopen  # b_soup_1.py
from bs4 import BeautifulSoup
import json
import pandas as pd

In [26]:
#bdc665fdd718f4f2624518fada5e39f8

In [27]:
html = urlopen('http://api.eia.gov/series/?api_key=bdc665fdd718f4f2624518fada5e39f8&series_id=EBA.US48-ALL.NG.H')

In [28]:
tg_raw = BeautifulSoup(html.read(), "lxml")

In [29]:
fout = open('tgraw_temp.txt', 'wt', encoding='utf-8')
fout.write(str(tg_raw))
fout.close()

In [30]:
tg_raw = str(tg_raw)
print(tg_raw)

<html><body><p>{"request":{"command":"series","series_id":"EBA.US48-ALL.NG.H"},"series":[{"series_id":"EBA.US48-ALL.NG.H","name":"Net generation for United States Lower 48 (region), hourly - UTC time","units":"megawatthours","f":"H","description":"Timestamps follow the ISO8601 standard (https:\/\/en.wikipedia.org\/wiki\/ISO_8601). Hourly representations are provided in Universal Time.","start":"20150701T05Z","end":"20200206T05Z","updated":"2020-02-06T08:59:39-0500","data":[["20200206T05Z",422292],["20200206T04Z",448953],["20200206T03Z",468866],["20200206T02Z",480134],["20200206T01Z",482455],["20200206T00Z",478744],["20200205T23Z",470874],["20200205T22Z",461285],["20200205T21Z",455621],["20200205T20Z",455343],["20200205T19Z",458452],["20200205T18Z",461905],["20200205T17Z",464275],["20200205T16Z",467654],["20200205T15Z",466165],["20200205T14Z",458206],["20200205T13Z",444897],["20200205T12Z",421574],["20200205T11Z",393222],["20200205T10Z",376082],["20200205T09Z",371952],["20200205T08Z",37

In [31]:
data_index = tg_raw.find("data")

In [32]:
tg_raw = tg_raw[data_index + 8: -23]

In [33]:
tg_raw = tg_raw.split('],[')
print(tg_raw[0])
print(len(tg_raw)) 
temp = '[' + tg_raw[0] + ']'
temp_list = json.loads(temp)
print(temp_list)
print(type(temp_list))

"20200206T05Z",422292
40345
['20200206T05Z', 422292]
<class 'list'>


In [34]:
res = [];
for i in range(len(tg_raw)):
    temp = '[' + tg_raw[i] + ']';
    temp_list = json.loads(temp);
    res.append(temp_list);

In [35]:
tgraw_df = pd.DataFrame(res) 
tgraw_df.to_excel("tg_raw.xlsx")

In [36]:
tg_clean = tgraw_df.rename(columns={0: 'UTC', 1:'TotalGeneration'});
split_data = tg_clean['UTC'].str.split("T", n = 1, expand = True)
tg_clean["Date"] = split_data[0];
tg_clean["Time"] = split_data[1].str.split("Z", n = 1, expand = True)[0]
tg_clean.drop(columns =["UTC"], inplace = True)
tg_clean.to_excel("tg_clean.xlsx")